# SEC 13F Data Load and Clean
This notebook provides a complete pipeline for downloading, parsing, and cleaning U.S. SEC Form 13F Institutional Holdings data.

Form 13F is filed quarterly by institutional investment managers with over $100 million in assets under management. It contains detailed holdings of U.S.-listed equities and related securities.

## 🔗 SEC official data link:
https://www.sec.gov/data-research/sec-markets-data/form-13f-data-sets

## 📥 To use this notebook:
Download the most recent quarterly dataset from the link above and execute the notebook locally.

## 📁 Dataset Documentation
This dataset is derived directly from the SEC's structured 13F data files. Each row represents a single security holding reported by an institutional filer as of the quarter-end date.

## ✅ Key Columns

Column Name	Type	Description
FILINGMANAGER_NAME	str	Name of the institutional investment manager who filed the 13F.
NAMEOFISSUER	str	Company name (issuer) of the reported security.
CUSIP	str	9-digit identifier for the security.
DATE	str	Reporting period end date (e.g., 31-DEC-2024).
NUMBER_OF_SHARES	int	Number of shares held by the filer.
NOTIONAL_VALUE	float	Market value of the holding in USD (reported in whole dollars as of 2023).
SUBMISSIONTYPE	str	Type of SEC filing: <ul><li>13F-HR: Initial holdings report</li><li>13F-HR/A: Amendment</li><li>13F-NT: Notice (no holdings disclosed)</li></ul>
CIK	str	Central Index Key – unique SEC ID for each institutional manager.
INVESTMENTDISCRETION	str	Discretion type:
SOLE, SHARED, or NONE.
OTHERMANAGER	str	Sequence number of additional managers (if shared discretion applies).
ACCESSION_NUMBER	str	Unique identifier for the SEC filing, useful for linking to original EDGAR text.

## 📌 Use Case Ideas
Track quarterly position changes of large asset managers using FILINGMANAGER_NAME + DATE

Identify top holders of a specific company via NAMEOFISSUER

Combine with shares outstanding data to estimate ownership percentage

Use CIK to link filings across years or with other SEC datasets (e.g., 13D/13G)

Distinguish active vs. passive ownership using INVESTMENTDISCRETION

## 📎 Data Source
This dataset is based on the official SEC Form 13F quarterly releases, provided in tab-delimited (TSV) flat file format:

SEC 13F Datasets - EDGAR </br>
https://www.sec.gov/data-research/sec-markets-data/form-13f-data-sets


In [1]:
import pandas as pd
import os

# Set the Load Path 
data_dir = ".../SecFiles/13F/01dec2024-28feb2025_form13f/"


In [ ]:

# Read 13F files 
submission_df = pd.read_csv(os.path.join(data_dir, "SUBMISSION.tsv"), sep="\t", dtype=str)
coverpage_df = pd.read_csv(os.path.join(data_dir, "COVERPAGE.tsv"), sep="\t", dtype=str)
infotable_df = pd.read_csv(os.path.join(data_dir, "INFOTABLE.tsv"), sep="\t", dtype=str)
summary_df = pd.read_csv(os.path.join(data_dir, "SUMMARYPAGE.tsv"), sep="\t", dtype=str)


In [19]:
# Examples 
print("📄 SUBMISSION:")
print(submission_df.head())

print("\n📄 COVERPAGE:")
print(coverpage_df[['ACCESSION_NUMBER', 'FILINGMANAGER_NAME']].head())

print("\n📄 INFOTABLE:")
print(infotable_df[['ACCESSION_NUMBER', 'NAMEOFISSUER', 'CUSIP', 'VALUE', 'SSHPRNAMT']].head())

print("\n📄 SUMMARYPAGE:")
print(summary_df.head())

📄 SUBMISSION:
       ACCESSION_NUMBER  FILING_DATE SUBMISSIONTYPE         CIK PERIODOFREPORT
0  0001088875-25-000002  31-JAN-2025         13F-HR  0001088875    31-DEC-2024
1  0001957363-25-000001  31-JAN-2025         13F-HR  0001957363    31-DEC-2024
2  0001088875-25-000004  31-JAN-2025         13F-NT  0001088876    31-DEC-2024
3  0001706836-25-000002  31-JAN-2025         13F-HR  0001706836    31-DEC-2024
4  0000922423-25-000009  31-JAN-2025         13F-HR  0001407024    31-DEC-2024

📄 COVERPAGE:
       ACCESSION_NUMBER                        FILINGMANAGER_NAME
0  0002031775-25-000001       BridgePort Financial Solutions, LLC
1  0001628280-25-008999        Provida Pension Fund Administrator
2  0002058285-25-000002          Park Square Financial Group, LLC
3  0001667731-25-000316  Total Wealth Planning & Management, Inc.
4  0001803156-25-000001                Menard Financial Group LLC

📄 INFOTABLE:
       ACCESSION_NUMBER    NAMEOFISSUER      CUSIP    VALUE SSHPRNAMT
0  0002031775-25-0

In [15]:
# Merge Data 
meta_df = pd.merge(submission_df[["ACCESSION_NUMBER", "PERIODOFREPORT", "SUBMISSIONTYPE", "CIK"]],
                   coverpage_df[["ACCESSION_NUMBER", "FILINGMANAGER_NAME"]],
                   on="ACCESSION_NUMBER", how="inner")

combined_df = pd.merge(infotable_df, meta_df, on="ACCESSION_NUMBER", how="inner")

# Rename 
final_df = combined_df[[
    "FILINGMANAGER_NAME",
    "NAMEOFISSUER",
    "CUSIP",
    "PERIODOFREPORT",
    "SSHPRNAMT",
    "VALUE",
    "SUBMISSIONTYPE",
    "CIK",
    "INVESTMENTDISCRETION",
    "OTHERMANAGER",
    "ACCESSION_NUMBER"
]].rename(columns={
    "PERIODOFREPORT": "DATE",
    "SSHPRNAMT": "NUMBER_OF_SHARES",
    "VALUE": "NOTIONAL_VALUE", 
})



In [16]:
final_df.columns

Index(['FILINGMANAGER_NAME', 'NAMEOFISSUER', 'CUSIP', 'DATE',
       'NUMBER_OF_SHARES', 'NOTIONAL_VALUE', 'SUBMISSIONTYPE', 'CIK',
       'INVESTMENTDISCRETION', 'OTHERMANAGER', 'ACCESSION_NUMBER'],
      dtype='object')

In [17]:
# dtype transform 
final_df["NUMBER_OF_SHARES"] = pd.to_numeric(final_df["NUMBER_OF_SHARES"], errors='coerce')
final_df["NOTIONAL_VALUE"] = pd.to_numeric(final_df["NOTIONAL_VALUE"], errors='coerce')

# show 
print(final_df.head())

                    FILINGMANAGER_NAME    NAMEOFISSUER      CUSIP  \
0  BridgePort Financial Solutions, LLC           3M CO  88579Y101   
1  BridgePort Financial Solutions, LLC     ABBOTT LABS  002824100   
2  BridgePort Financial Solutions, LLC    ALPHABET INC  02079K305   
3  BridgePort Financial Solutions, LLC    ALPHABET INC  02079K107   
4  BridgePort Financial Solutions, LLC  AMAZON COM INC  023135106   

          DATE  NUMBER_OF_SHARES  NOTIONAL_VALUE SUBMISSIONTYPE         CIK  \
0  31-DEC-2024              4073          613102         13F-HR  0002031775   
1  31-DEC-2024              2171          295022         13F-HR  0002031775   
2  31-DEC-2024              7837         1320550         13F-HR  0002031775   
3  31-DEC-2024              3896          663126         13F-HR  0002031775   
4  31-DEC-2024              9660         2016335         13F-HR  0002031775   

  INVESTMENTDISCRETION OTHERMANAGER      ACCESSION_NUMBER  
0                 SOLE            0  0002031775-25

In [12]:
import re
import pandas as pd
from rapidfuzz import process, fuzz

# Cleaning function: convert to lowercase and remove common legal suffixes
def clean_name(name):
    name = name.lower()
    name = re.sub(r'\b(lp|llc|l\.p\.|l\.p|inc\.|inc|master|fund|limited|partners|series|management|capital|advisors|group|asset|holdings|co|corp|company)\b', '', name)
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    name = re.sub(r'\s+', ' ', name).strip()  # Remove extra spaces
    return name

# Clean both fund name lists
cleaned_firm_list = [(name, clean_name(name)) for name in Firm_fund_name_list]
cleaned_f13_list = [(name, clean_name(name)) for name in F13_fund_name_list]

# Perform fuzzy matching
results = []

for firm_original, firm_clean in cleaned_firm_list:
    match, score, _ = process.extractOne(
        firm_clean, [f13[1] for f13 in cleaned_f13_list], scorer=fuzz.token_sort_ratio
    )
    matched_f13_original = next(orig for orig, cleaned in cleaned_f13_list if cleaned == match)
    results.append({
        'Firm_fund_name': firm_original,
        'Matched_F13_name': matched_f13_original,
        'Similarity_Score': score
    })

# Convert result to DataFrame
df_results = pd.DataFrame(results)
